In [1]:
import numpy as np
import pandas
import pm4py
from matplotlib import pyplot as plt
from sklearn.mixture import GaussianMixture
import scipy.stats as stats
import ot
import os
from tqdm import tqdm
import collections
import matplotlib.dates as md
import importlib
import pickle
import random
import math
import CRPS.CRPS as pscore
import datetime

pandas.set_option('display.max_columns', None)
#pandas.set_option('display.max_rows', None)


import sys
sys.path.append('../../TaskExecutionTimeMining/')
from drbart_parser import *
from event_log_transformer import *

#sys.path.append('../../Evaluation')
sys.path.append('../../Evaluation/')
from normal_evaluation.drbart_evaluation import *

In [2]:
log_name = 'test'
with open('../transformed_event_logs/BPIC_2017_all_'+log_name+'.pickle', 'rb') as f:
    test_event_log = pickle.load(f)

test_event_log['case:concept:name'] = test_event_log['case:concept:name'].astype(str)
known_resources = ['User_1','User_10','User_100','User_101','User_102','User_103','User_104','User_105','User_106','User_107','User_108','User_109','User_11','User_110','User_111','User_112','User_113','User_114','User_115','User_116','User_117','User_118','User_119','User_12','User_120','User_121','User_122','User_123','User_124','User_125','User_126','User_127','User_128','User_129','User_13','User_130','User_131','User_132','User_133','User_134','User_135','User_136','User_137','User_138','User_139','User_14','User_140','User_141','User_142','User_143','User_144','User_145','User_146','User_147','User_148','User_149','User_15','User_16','User_17','User_18','User_19','User_2','User_20','User_21','User_22','User_23','User_24','User_25','User_26','User_27','User_28','User_29','User_3','User_30','User_31','User_32','User_33','User_34','User_35','User_36','User_37','User_38','User_39','User_4','User_40','User_41','User_42','User_43','User_44','User_45','User_46','User_47','User_48','User_49','User_5','User_50','User_51','User_52','User_53','User_54','User_55','User_56','User_57','User_58','User_59','User_6','User_60','User_61','User_62','User_63','User_64','User_65','User_66','User_67','User_68','User_69','User_7','User_70','User_71','User_72','User_73','User_74','User_75','User_76','User_77','User_78','User_79','User_8','User_80','User_81','User_82','User_83','User_84','User_85','User_86','User_87','User_88','User_89','User_9','User_90','User_91','User_92','User_93','User_94','User_95','User_96','User_97','User_98','User_99']
known_activities = ['W_Assess potential fraud__ate_abort','W_Assess potential fraud__complete','W_Assess potential fraud__resume','W_Assess potential fraud__schedule','W_Assess potential fraud__start','W_Assess potential fraud__suspend','W_Assess potential fraud__withdraw','W_Call after offers__ate_abort','W_Call after offers__complete','W_Call after offers__resume','W_Call after offers__schedule','W_Call after offers__start','W_Call after offers__suspend','W_Call after offers__withdraw','W_Call incomplete files__ate_abort','W_Call incomplete files__complete','W_Call incomplete files__resume','W_Call incomplete files__schedule','W_Call incomplete files__start','W_Call incomplete files__suspend','W_Complete application__ate_abort','W_Complete application__complete','W_Complete application__resume','W_Complete application__schedule','W_Complete application__start','W_Complete application__suspend','W_Handle leads__complete','W_Handle leads__resume','W_Handle leads__schedule','W_Handle leads__start','W_Handle leads__suspend','W_Handle leads__withdraw','W_Shortened completion __resume','W_Shortened completion __schedule','W_Shortened completion __start','W_Shortened completion __suspend','W_Validate application__ate_abort','W_Validate application__complete','W_Validate application__resume','W_Validate application__schedule','W_Validate application__start','W_Validate application__suspend']

/tmp/ipykernel_1916609/3301179562.py:3: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  test_event_log = pickle.load(f)


In [3]:
N = 1000
import conduct_evaluation
get_pscores = lambda likelihoods : [pscore(likelihoods[1][i], likelihoods[2][k][3]).compute()[0] for i, k in enumerate(list(likelihoods[0].keys()))]

In [4]:
drbart_model_A = DRBART(parser_dir = '../../../models/bpic_2017_all/concept-name/',
                     strict_parser=True)
evaluator_A = conduct_evaluation.ConductEvaluation(drbart_model_A, SampleOutcomes_DRBART_Normal_A, {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource_start',
                                                    },
                                     test_event_log, n=N)
likelihoods_A = evaluator_A.sample_cases(False, False)

  0%|                                                                                                                        | 0/6018 [00:00<?, ?it/s]


OverflowError: timestamp out of range for platform time_t

In [ ]:
sum([v.ln() for v in likelihoods_A[0].values()])

In [ ]:
np.mean(get_pscores(likelihoods_A))

In [ ]:
drbart_model_A_R = DRBART(parser_dir = '../../../models/bpic_2017_all/concept-name_resource/',
                     strict_parser=True)
evaluator_A_R = conduct_evaluation.ConductEvaluation(drbart_model_A_R, SampleOutcomes_DRBART_Normal_A_R,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                    },
                                     test_event_log, n=N)
likelihoods_A_R = evaluator_A_R.sample_cases(False, False)

In [ ]:
np.mean([v.ln() for v in likelihoods_A_R[0].values()])

In [ ]:
np.mean(get_pscores(likelihoods_A_R))

In [ ]:
drbart_model_R = DRBART(parser_dir = '../../../models/bpic_2017_all/resource/',
                     strict_parser=True)
evaluator_R = conduct_evaluation.ConductEvaluation(drbart_model_R, SampleOutcomes_DRBART_Normal_R,
                                                   {
                                                        'activity_key' : 'concept:name_start',
                                                        'resource_key' : 'org:resource_start',
                                                    },
                                     test_event_log, n=N)
likelihoods_R = evaluator_R.sample_cases(False, True)

In [ ]:
np.mean([v.ln() for v in likelihoods_R[0].values()])

In [ ]:
np.mean(get_pscores(likelihoods_R))

In [5]:
drbart_model_R_A_S = DRBART(parser_dir = '../../../models/bpic_2017_all/concept-name_resource_seconds-in-day/',
                     strict_parser=True)
evaluator_R_A_S = conduct_evaluation.ConductEvaluation(drbart_model_R_A_S, SampleOutcomes_DRBART_Normal_R_A_S,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource_start',
                                                    },
                                     test_event_log, n=N)
likelihoods_R_A_S = evaluator_R_A_S.sample_cases(False, False)

  0%|                                                                                                                        | 0/6018 [00:01<?, ?it/s]


KeyError: 'concept:name_start'

In [ ]:
np.mean([v.ln() for v in likelihoods_A_R_S[0].values()])

In [ ]:
np.mean(get_pscores(likelihoods_A_R_S))

In [ ]:
drbart_model_R_A_S_AC = DRBART(parser_dir = '../../../models/bpic_2017_all/concept-name_resource_seconds-in-day_activity-count/',
                     strict_parser=True)
evaluator_R_A_S_AC = conduct_evaluation.ConductEvaluation(drbart_model_R_A_S_AC, SampleOutcomes_DRBART_Normal_R_A_S_AC,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource_start',
                                                        'known_activities' : known_activities,
                                                        'max_sample' : 1000,
                                                    },
                                     test_event_log, n=1)
likelihoods_R_A_S_AC = evaluator_R_A_S_AC.sample_cases(False, False)

In [ ]:
np.mean([v.ln() for v in likelihoods_A_R_S_AC[0].values()])

In [ ]:
np.mean(get_pscores(likelihoods_A_R_S_AC))

In [ ]:
drbart_model_R_A_S_RC = DRBART(parser_dir = '../../../models/bpic_2017_all/concept-name_resource_seconds-in-day_resource-count/',
                     strict_parser=True)
evaluator_R_A_S_RC = conduct_evaluation.ConductEvaluation(drbart_model_R_A_S_RC, SampleOutcomes_DRBART_Normal_R_A_S_RC,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource_start',
                                                    },
                                     test_event_log, n=N)
likelihoods_R_A_S_RC = evaluator_R_A_S_RC.sample_cases(False, True)

In [ ]:
drbart_model_R_A_S_AC_RC = DRBART(parser_dir = '../../../models/bpic_2017_all/concept-name_resource_seconds-in-day_activitiy-count_resource-count/',
                     strict_parser=True)
evaluator_R_A_S_AC_RC = conduct_evaluation.ConductEvaluation(drbart_model_R_A_S_AC_RC, SampleOutcomes_DRBART_Normal_R_A_S_AC_RC,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource_start',
                                                    },
                                     test_event_log, n=N)
likelihoods_R_A_S_AC_RC = evaluator_R_A_S_AC_RC.sample_cases(False, True)

In [ ]:
np.mean([v.ln() for v in likelihoods_R_A_S_AC_RC[0].values()])

In [ ]:
np.mean(get_pscores(likelihoods_R_A_S_AC_RC))

In [ ]:
drbart_model_R_A_S_D = DRBART(parser_dir = '../../../models/bpic_2017_all/concept-name_resource_seconds-in-day_day-in-week/',
                     strict_parser=True)
evaluator_R_A_S_D = conduct_evaluation.ConductEvaluation(drbart_model_R_A_S_D, SampleOutcomes_DRBART_Normal_R_A_S_D,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource_start',
                                                    },
                                     test_event_log, n=N)
likelihoods_R_A_S_D = evaluator_R_A_S_D.sample_cases(False, True)

In [ ]:
np.mean([v.ln() for v in likelihoods_R_A_S_D[0].values()])

In [ ]:
np.mean(get_pscores(likelihoods_R_A_S_D))

In [ ]:
drbart_model_R_A_S_D_AC_RC = DRBART(parser_dir = '../../../models/bpic_2017_all/concept-name_resource_seconds-in-day_day-of-week_activity-count_resoure-count/',
                     strict_parser=True)
evaluator_R_A_S_D_AC_RC = conduct_evaluation.ConductEvaluation(drbart_model_R_A_S_D_AC_RC, SampleOutcomes_DRBART_Normal_R_A_S_D_AC_RC,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource_start',
                                                    },
                                     test_event_log, n=N)
likelihoods_R_A_S_D_AC_RC = evaluator_R_A_S_D_AC_RC.sample_cases(False, True)

In [ ]:
np.mean([v.ln() for v in likelihoods_R_A_S_D_AC_RC[0].values()])

In [ ]:
np.mean(get_pscores(likelihoods_R_A_S_D_AC_RC))

In [ ]:
results = {
    'drbart_model_A' : likelihoods_A,
    'drbart_model_A_S' : likelihoods_A_S,
    'drbart_model_A_S_D' : likelihoods_A_S_D,
    'drbart_model_A_S_AC' : likelihoods_A_S_AC
}
with open('./pcr_dr_bart_evaluation_'+log_name+'.pickle', 'wb') as handle:
    pickle.dump(results, handle)

In [ ]:
with open('./pcr_dr_bart_evaluation_'+log_name+'.pickle', 'rb') as handle:
    results = pickle.load(handle)

In [ ]:
likelihoods_A_S_D = results['drbart_model_A_S_D']